In [133]:
import numpy as np
import tensorflow as tf
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import seaborn as sns
#from datetime import datetime

In [134]:
df = pd.read_csv('../datas/train_passengers_copy.csv')
#Variables for training
cols = list(df)[1:3]
df_for_training = df[cols].astype('float64')

In [135]:
#LSTM uses sigmoid and tanh that are sensitive to magnitude so values need to be normalized
# normalize the dataset
scaler = MinMaxScaler()
scaler.fit(df_for_training)
df_for_training_scaled = scaler.transform(df_for_training)

In [136]:
#As required for LSTM networks, we require to reshape an input data into n_samples x timesteps x n_features. 
#In this example, the n_features is 5. We will make timesteps = 14 (past days data used for training). 

#Empty lists to be populated using formatted training data
trainX = []
trainY = []

n_future = 1   # Number of days we want to look into the future based on the past days.
n_past = 3  # Number of past days we want to use to predict the future.

In [137]:
#Reformat input data into a shape: (n_samples x timesteps x n_features)
#In my example, my df_for_training_scaled has a shape (12823, 5)
#12823 refers to the number of data points and 5 refers to the columns (multi-variables).
for i in range(n_past, len(df_for_training_scaled) - n_future +1):
    trainX.append(df_for_training_scaled[i - n_past:i, 0:df_for_training.shape[1]])
    trainY.append(df_for_training_scaled[i + n_future - 1:i + n_future, 0])

trainX, trainY = np.array(trainX), np.array(trainY)

print('trainX shape == {}.'.format(trainX.shape))
print('trainY shape == {}.'.format(trainY.shape))

trainX shape == (141, 3, 2).
trainY shape == (141, 1).


In [138]:
test_model = tf.keras.models.load_model("../saved_param/test_model1/")
weights_test_model_1 = test_model.layers[0].get_weights()

In [139]:
model = tf.keras.layers.LSTM(3, input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=True, return_state=True)
init_weights = model(np.array([trainX[1]])) # you need to pass an input to initialize the weights then setup the weights manually

In [140]:
model.set_weights(weights_test_model_1)

In [141]:
model.get_weights()

[array([[ 0.28213406,  0.40180504,  0.10323656,  0.49266553,  0.59737396,
         -0.27416527, -0.6140406 , -0.22036755, -0.36234343,  0.11618108,
         -0.40446782, -0.05731392],
        [ 0.09265673, -0.55035496, -0.0250091 ,  0.10383672, -0.03438932,
          0.44054282, -0.02767795,  0.35591948, -0.076204  , -0.6510474 ,
         -0.4240626 , -0.43181473]], dtype=float32),
 array([[-0.11440527, -0.09116384,  0.10766367,  0.41458526,  0.13482785,
          0.02425341, -0.14716567,  0.09884649, -0.07846723, -0.2423919 ,
          0.7091871 ,  0.42080373],
        [-0.09434418,  0.6440063 , -0.16892694,  0.09986253, -0.29395232,
         -0.35163632, -0.18524946,  0.01135916, -0.2985677 ,  0.35922354,
          0.23562133, -0.1401508 ],
        [ 0.12497412,  0.13285607,  0.29592115, -0.17216626, -0.05532025,
         -0.09946106, -0.70869935,  0.2606318 ,  0.50013614, -0.11444857,
         -0.03364325, -0.04491055]], dtype=float32),
 array([0., 0., 0., 1., 1., 1., 0., 0., 0., 0.

In [142]:
model(np.array([trainX[1]]))

[<tf.Tensor: shape=(1, 3, 3), dtype=float32, numpy=
 array([[[-0.00752289,  0.04412096, -0.0144942 ],
         [-0.01602702,  0.06958452, -0.02986364],
         [-0.01964575,  0.08610261, -0.04356184]]], dtype=float32)>,
 <tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[-0.01964575,  0.08610261, -0.04356184]], dtype=float32)>,
 <tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[-0.0476364 ,  0.2013372 , -0.10137359]], dtype=float32)>]

In [143]:
trainX[1]

array([[0.02702703, 0.8       ],
       [0.05405405, 0.6       ],
       [0.04826255, 0.6       ]])

In [148]:
model.states

ListWrapper([None, None])

In [144]:
cell_model = tf.keras.layers.LSTMCell(3, input_shape=(trainX.shape[1], trainX.shape[2]))

In [145]:
cell_model.build(input_shape=(trainX.shape[1], trainX.shape[2]))
cell_model(trainX[1])

TypeError: call() missing 1 required positional argument: 'states'